# Fang & Oosterle (2008) : A novel pricing method for european options based on Fourier-Cosine series expansions (Part I/II) 

In [2]:
#TODO:
# implement (22) & (23)  which are needed in (29)

# implement pricing formula (30), and all needed components

# implement Heston model

# replicate numerical results (5.)

Formulas and text are from Fang & Oosterle 2008 paper. Chapter and equation numbers also correspond)

In [3]:
%matplotlib inline

import cmath   # c stands for complex (to handle complex numbers)
import numpy as np
import matplotlib.pyplot as plt

## 3. Pricing European Options

COS formula for general underlying processes:
<img src="figures/f_19.png" width="680">

### 3.1. Coeﬃcients V k for Plain Vanilla Options
Before we can use (19) for pricing options, the payoﬀ series coeﬃcients, $V_k$ , have to be recovered. We can ﬁnd analytic solutions for $V_k$ for several contracts (Fang & Ooesterle, 2008)

Both the cosine series coeﬃcients, $\chi_k$ , of $g(y) = e^y$ on $[c,\ d] ⊂ [a,\ b]$ and 

the cosine series coeﬃcients, $\psi_k$, of $g(y) = 1$ on $[c,\ d] ⊂ [a,\ b]$, are known analytically

<img src="figures/f_22.png" width="480">

<img src="figures/f_23.png" width="480">

In [5]:
# equation (22)
def chi_cosine_series_coeff(k, a_param, b_param, c_param, d_param):
    """
    Function (9) in the F&O paper
    Args:
        k : coeﬃcients of the summand (runs from 0 to N-1)
        a_param : float, left border of the trucated integral
        b_param : float, right border of the truncated integral
        c_param : 
        d_param : 
        
        
    Returns:
        chi_k : float, a cosine series coefficient
    """

    
# equation (23)
def psi_cosine_series_coeff(k, a_param, b_param, c_param, d_param):
    """
    Function (9) in the F&O paper
    Args:
        k : coeﬃcients of the summand (runs from 0 to N-1)
        a_param : float, left border of the trucated integral
        b_param : float, right border of the truncated integral
        c_param : 
        d_param : 
        
    Returns:
        psi_k : float, a cosine series coefficient
    """

### 3.3. Formula for Lévy Processes and the Heston Model

__Lévy Processes__
<img src="figures/f_27_30.png" width="480">

For the CGMY/KoBol model, which encompasses the __Geometric Brownian Motion (GBM)__ and __Variance Gamma (VG)__ models, the characteristic function of the log-asset price is of the form:

<img src="figures/f_31.png" width="480">

In [7]:
def U_cosine_coeff(k, a_param, b_param, call):
    """
    equation (29)
    """    
    # if the option is a call
    if call:
        chi_k = chi_cosine_series_coeff(k, a_param, b_param, 0, b_param) 
        psi_k = psi_cosine_series_coeff(k, a_param, b_param, 0, b_param)

        return (2 / (b_param - a_param)) * (chi_k - psi_k)
        
    # if the option is a put
    else:
        chi_k = chi_cosine_series_coeff(k, a_param, b_param, a_param, 0) 
        psi_k = psi_cosine_series_coeff(k, a_param, b_param, a_param, 0)
        
        return (2 / (b_param - a_param)) * (-chi_k + psi_k)

Note SWS: We can define $F_k$ analoguous to the density case, so that we only need to run it $N$ times instead for every x. Thus,
$$
F_k = \phi\left(\frac{k\pi}{b-a}\right) \ U_k    \qquad with \qquad \phi = some\ characteristic \ function
$$

In [8]:
def get_Fk_values(a_param, b_param, char_fun, N, call_flag):
    """
    We only want to do this once and not for each x
    Args:
        a : float, left border of the trucated integral, e.g. mean-6*std
        b : float, right border of the truncated integral, e.g. mean+6*std
        char_fun : a characteristic function, taking 1 argument (\omega)
        N: int, should be between 50 and 500
        call : bool 
    Returns:
       np.array of shape (N,) containing the values computed by Fk for each k 
    """
    # This is a numpy placeholder for our values obtained by Fk
    values_Fk = np.zeros(N)
    
    # here we fill our placeholder with the correct values of Fk for each k
    for k in range(N):
        
        U_k = U_cosine_coeff(k, a_param, b_param, call=call_flag) 
        
        omega = (k*cmath.pi) / (b_param-a_param)
        
        values_Fk[k] = char_fun(omega) * U_k
        
    return values_Fk

__Heston Model__

In the Heston model, the volatility, denoted by $\sqrt{u_t}$, is modeled by a stochastic diﬀerential equation:
<img src="figures/f_32.png" width="480">
<img src="figures/f_33_34.png" width="480">
<img src="figures/f_33b.png" width="480">

## 5. Numerical Results
### 5.1. Truncation Range for COS Method